In [6]:
import numpy as np
import re

def parse_mma_dump(filename):
    """
    Parses kernel printf dump of form:

    lane  0 | A: [ a a | a a | a a | a a ]
              B: [ b b | b b ]
              C: [ c c c c ]

    Returns:
        A: (32, 4, 2)
        B: (32, 2, 2)
        C: (32, 4)
    """

    A = np.zeros((32, 4, 2), dtype=np.float32)
    B = np.zeros((32, 2, 2), dtype=np.float32)
    C = np.zeros((32, 4), dtype=np.float32)

    lane_re = re.compile(r"lane\s+(\d+)")
    float_re = re.compile(r"[-+]?\d*\.\d+|\d+")

    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # lane id
            m = lane_re.search(line)
            if not m:
                continue

            lane = int(m.group(1))

            # extract all floats in order
            vals = [float(x) for x in float_re.findall(line)]

            # Expected order:
            # A: 8 floats
            # B: 4 floats
            # C: 4 floats
            #
            # total = 16
            vals = vals[1:]
            if len(vals) != 16:
                raise ValueError(
                    f"Lane {lane}: expected 16 floats, got {len(vals)}"
                )

            # A
            A[lane, 0, 0] = vals[0]
            A[lane, 0, 1] = vals[1]

            A[lane, 1, 0] = vals[2]
            A[lane, 1, 1] = vals[3]

            A[lane, 2, 0] = vals[4]
            A[lane, 2, 1] = vals[5]

            A[lane, 3, 0] = vals[6]
            A[lane, 3, 1] = vals[7]

            # B
            B[lane, 0, 0] = vals[8]
            B[lane, 0, 1] = vals[9]

            B[lane, 1, 0] = vals[10]
            B[lane, 1, 1] = vals[11]

            # C
            C[lane, 0] = vals[12]
            C[lane, 1] = vals[13]
            C[lane, 2] = vals[14]
            C[lane, 3] = vals[15]

    return A, B, C


In [7]:
A,B,C = parse_mma_dump("data.txt")

In [8]:
print(A.shape)
print(B.shape)
print(C.shape)

(32, 4, 2)
(32, 2, 2)
(32, 4)


In [9]:
A_new = np.copy(A)

In [13]:
A_new = np.transpose(A_new,(0,2,1))
print(A_new.shape)

(32, 4, 2)


In [17]:
A_new = A_new.reshape((8,8,4))
A_new = np.transpose(A_new, (2,0,1))
print(A_new.shape)

(4, 8, 8)


In [18]:
print(A_new)

[[[0.01   0.5117 1.0078 1.5078 2.0156 2.5156 3.0156 3.5156]
  [4.     4.5    5.     5.5    6.     6.5    7.     7.5   ]
  [0.02   0.5195 1.0234 1.5234 2.0156 2.5156 3.0156 3.5156]
  [4.0312 4.5312 5.0312 5.5312 6.0312 6.5312 7.0312 7.5312]
  [0.03   0.5312 1.0312 1.5312 2.0312 2.5312 3.0312 3.5312]
  [4.0312 4.5312 5.0312 5.5312 6.0312 6.5312 7.0312 7.5312]
  [0.04   0.5391 1.0391 1.5391 2.0469 2.5469 3.0469 3.5469]
  [4.0312 4.5312 5.0312 5.5312 6.0312 6.5312 7.0312 7.5312]]

 [[0.05   0.5508 1.0469 1.5469 2.0469 2.5469 3.0469 3.5469]
  [4.0625 4.5625 5.0625 5.5625 6.0625 6.5625 7.0625 7.5625]
  [0.0601 0.5586 1.0625 1.5625 2.0625 2.5625 3.0625 3.5625]
  [4.0625 4.5625 5.0625 5.5625 6.0625 6.5625 7.0625 7.5625]
  [0.0698 0.5703 1.0703 1.5703 2.0625 2.5625 3.0625 3.5625]
  [4.0625 4.5625 5.0625 5.5625 6.0625 6.5625 7.0625 7.5625]
  [0.     0.1602 0.3203 0.4805 0.6406 0.8008 0.9609 1.1172]
  [1.2812 1.4375 1.6016 1.7578 1.9219 2.0781 2.2344 2.4062]]

 [[0.2598 0.7617 1.2578 1.7578 2.265